## Mesh open3d用例

### 1. Mesh 数据读取

open3d 支持从 `ply`, `stl`, `obj`, `off`, `gltf/glb` 等格式的 mesh 读取。


PLY数据
> 一个典型的PLY对象定义只是一个顶点的(x,y,z)三元组列表和一个由顶点列表的索引描述的面的列表。大多数PLY文件包括这些核心信息。顶点和面是 "元素 "的两个例子，而PLY文件的主要内容是它的元素列表。
>
> 在一个给定的文件中，每个元素都有一个固定数量的 "属性"，这些属性是为每个元素指定的。PLY文件中的典型信息只包含两个元素，即顶点的（x,y,z）三元组和每个面的顶点指数。

In [31]:
# 读取 mesh
import open3d as o3d
import numpy as np

filePath = "../data/pz.STL"
mesh = o3d.io.read_triangle_mesh(filePath)
# mesh.compute_vertex_normals()

print(mesh)
print('Vertices:')
print(np.asarray(mesh.vertices))
print('Triangles:')
print(np.asarray(mesh.triangles))

TriangleMesh with 1444 points and 908 triangles.
Vertices:
[[ 1.5199995   0.          9.23999977]
 [ 1.62529016  0.          7.96932936]
 [15.33216381  0.         13.98172188]
 ...
 [ 1.80283511 79.77000427 10.48104286]
 [ 1.69999945 79.77000427  9.23999977]
 [ 1.69999945 74.20000458  9.23999977]]
Triangles:
[[   0    1    2]
 [   3    4    1]
 [   1    4    5]
 ...
 [1436 1438 1439]
 [1440 1441 1442]
 [1440 1442 1443]]


此时的可视化结果，只有灰度，没有计算表面法向量。  因此缺乏立体感

In [32]:
# 可视化
print("Try to render a mesh with normals (exist: " +
      str(mesh.has_vertex_normals()) + ") and colors (exist: " +
      str(mesh.has_vertex_colors()) + ")")
o3d.visualization.draw_geometries([mesh])
print("A mesh with no normals and no colors does not look good.")


Try to render a mesh with normals (exist: True) and colors (exist: False)
A mesh with no normals and no colors does not look good.


### 2. 法向量计算
读取 `ply` 生成的 mesh，初始法向量为 `empty`.


In [33]:
# 表面法线计算
print("Computing normal and rendering it.")
print("\n原始法向量：")
print(np.asarray(mesh.triangle_normals))
mesh.compute_vertex_normals()
print("\n法向量估计结果：")
print(np.asarray(mesh.triangle_normals))
o3d.visualization.draw_geometries([mesh])


Computing normal and rendering it.

原始法向量：
[]

法向量估计结果：
[[ 0.         -1.          0.        ]
 [-0.         -1.          0.        ]
 [ 0.         -1.          0.        ]
 ...
 [-0.96940019  0.          0.2454858 ]
 [-0.9965845   0.          0.08257926]
 [-0.9965845   0.          0.08257926]]


### 3. Mesh 数据复制
利用 `numpy` 拷贝 mesh, 选取前一半的点，或者随机采样。


In [34]:
import copy

# # 选取前一半的点
# print("We make a partial mesh of only the first half triangles.")
# mesh1 = copy.deepcopy(mesh)
# mesh1.triangles = o3d.utility.Vector3iVector(
#     np.asarray(mesh1.triangles)[:len(mesh1.triangles) // 2, :])
# mesh1.triangle_normals = o3d.utility.Vector3dVector(
#     np.asarray(mesh1.triangle_normals)[:len(mesh1.triangle_normals) // 2, :])
# print(mesh1.triangles)
# o3d.visualization.draw_geometries([mesh1])
#
# # 随机采样 1/2
# mesh1 = copy.deepcopy(mesh)
# # 采样点下标
# choice = np.random.choice(range(len(mesh1.triangle_normals)),
#     len(mesh1.triangle_normals)//2, replace=False)
# mesh1.triangles = o3d.utility.Vector3iVector(
#     np.asarray(mesh1.triangles)[choice, :])
# mesh1.triangle_normals = o3d.utility.Vector3dVector(
#     np.asarray(mesh1.triangle_normals)[choice, :])
# print(mesh1.triangles)
# o3d.visualization.draw_geometries([mesh1])
#


### 4. Mesh 点云采样

#### 4.1. uniformly 根据每个三角形面积均匀采样

In [36]:
# o3d.visualization.draw_geometries([mesh])
pcd = mesh.sample_points_uniformly(number_of_points=5000)
o3d.visualization.draw_geometries([pcd])


#### 4.2. poisson 曲面均匀分布采样

`init_factor` 是预选点云倍数，如采样 n 点、init_factor=5，则预选 5n 点云，进行poisson采样。

In [37]:
pcd = mesh.sample_points_poisson_disk(number_of_points=2500, init_factor=5)
o3d.visualization.draw_geometries([pcd])


### 5. Mesh 网格细分


### 6. Mesh 网格简化

